In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# from transformers import AutoTokenizer, AutoModel
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

model_dir='/opt/cache/modelscope/hub/qwen/Qwen1___5-7B-Chat'
# Now you do not need to add "trust_remote_code=True"
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def evaluate(model,prompt):
    messages = [
        # {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)
    print('-------')
    # return response

In [3]:
evaluate(model, "老板")
evaluate(model ,  "你好，我想开发票")
evaluate(model ,  "你好！买书有增值税发票吗？")
evaluate(model ,  "你好我想补开发票")
evaluate(model ,  "怎么看到我的发票")
evaluate(model ,  "发票开好了吗")
evaluate(model ,  "你们发什么快递？")
evaluate(model ,  "你知道哪些快递公司？")

老板通常是指在企业中拥有管理和决策权力的人，他们负责制定公司战略、管理日常运营、分配资源、设定目标并监督员工工作。你可以提供更具体的问题或者需要关于哪位老板的信息，我会尽力帮你解答。
-------
您好！如果您需要开发票，具体的流程可能会根据您所在的地区和使用的业务环境有所不同，但一般来说，可以按照以下步骤进行：

1. **确认购买或服务**：首先，您需要确认您已经完成了商品或服务的交易，有相应的消费记录或者服务提供单据。

2. **索取发票信息**：向提供商品或服务的商家索要发票信息，包括但不限于发票抬头（如果是公司，通常是公司的全称和税号）、购买日期、商品或服务名称、数量、单价等详细内容。

3. **填写发票**：如果您是商家，可能需要通过后台系统生成电子发票；如果是消费者，可能需要填写税务局提供的发票申请表。

4. **支付费用**：如果需要打印纸质发票，可能需要支付一定的打印或邮寄费用。如果是电子发票，一般会直接发送到您的邮箱。

5. **保存发票**：确保妥善保管好发票，以便后续可能的税务申报或报销。

6. **提交或打印**：最后，将填好的发票信息提交给税务局或者打印出来保留。

具体的操作步骤，建议您咨询您所在地区的税务局或者商家，他们会提供更准确和详细的指导。希望这个信息对您有所帮助！
-------
您好！购买书籍通常情况下是可以获得增值税专用发票的。书店、电商平台等正规卖家在销售图书时，如果按照我国税法规定，会对商品征收增值税，因此会提供增值税专用发票以供消费者抵扣。但具体情况可能会因地区和商家政策而异，建议在购买时向卖家确认一下。
-------
您好，如果您需要补开发票，通常需要提供以下几个信息或者步骤：

1. **购买凭证**：首先，您需要提供购买商品或服务的原始交易凭证，比如收据、订单号或者付款记录等。这些信息能帮助商家确认您的购买行为。

2. **联系商家**：找到当初为您开具发票的商家，可以通过电话、邮件或者实体店进行咨询。告诉他们您需要补开发票，并提供相关凭证。

3. **填写申请**：商家可能会要求您填写一份补开发票的申请表，包括购买日期、商品信息、金额等详细信息。

4. **支付差额**：如果原发票丢失或者损坏，可能需要支付一定的手续费，以补偿重新开具的成本。

5. **等待回复**：商家收到申请后，会进

In [4]:
from peft import PeftModel
new_model = PeftModel.from_pretrained(
            model,
            "/opt/data/qwen1.5-7b/customer_sft_5",
            device_map='auto'
        )

In [5]:
evaluate(new_model ,  "老板")
evaluate(new_model ,  "你好，我想开发票")
evaluate(new_model ,  "你好！买书有增值税发票吗？")
evaluate(new_model ,  "你好我想补开发票")
evaluate(new_model ,  "怎么看到我的发票")
evaluate(new_model ,  "发票开好了吗")
evaluate(new_model ,  "你们发什么快递？")
# evaluate(new_model ,  "你好我想补开发票")
# evaluate(new_model ,  "不要发京东，这边京东刚倒闭了")
# evaluate(new_model ,  "什么时候发货啊")
# evaluate(new_model ,  "那我能退货吗")
# evaluate(new_model ,  "可以换货吗？")
# evaluate(new_model ,  "可以包邮吗")
# evaluate(new_model ,  "正版吗？")
# evaluate(new_model ,  "这个是新书 还是二手书？")
# evaluate(new_model ,  "有多少库存")
# evaluate(new_model ,  "有目录吗")
# evaluate(new_model ,  "从那发货")
# evaluate(new_model ,  "包装好一些")

您好！有什么可以帮到您的吗？如果您是想要询问关于工作、管理或者其他与老板相关的问题，欢迎继续提出。
-------
您好！如果您是商家或需要开发票，这通常涉及到以下几个步骤：

1. **购买商品/服务**：首先，您需要确保已经购买了商品或者接受了服务。如果是线上购物，通常在支付页面会自动提供发票选项；如果是线下交易，记得保留收据。

2. **索取发票信息**：向商家索要发票时，一般需要提供您的姓名、联系方式、购买的商品或服务名称、数量、价格等详细信息。如果是在电商平台，可以在订单详情页申请。

3. **填写发票信息**：如果是自填的纸质发票，需要填写公司名称（如有）、税号、地址等税务信息。如果是电子发票，系统会自动生成。

4. **支付费用**：如果需要额外付费才能获取发票，比如餐饮行业的小费，这时就需要支付相应的费用。

5. **等待发票寄送**：商家处理完后，发票会通过快递或者电子邮件等方式寄送到您指定的地址。

6. **保存和管理**：收到发票后，请妥善保管，以备查用。有些地方可能需要定期提交给税务局。

请注意，不同地区和行业的发票要求可能会有所不同，具体操作请参照当地的规定。如果您遇到问题，可以联系商家或当地的税务机构寻求帮助。
-------
您好！购买书籍一般情况下是可以获得增值税专用发票的。书店、电商平台等正规卖家在销售图书时，如果按照我国税法的要求，会对商品征收增值税，并向消费者开具增值税专用发票。这种发票可以作为税务报销或抵扣的凭证。但具体是否能开具增值税发票，建议您在购买时向卖家咨询确认，以确保信息的准确性。
-------
您好，很高兴为您提供帮助。但是，作为一个AI助手，我无法直接处理具体的发票补开发票事宜，因为这需要依赖于具体的业务系统或税务机关的规定。一般来说，您需要提供相关的购买信息，如购买日期、商品信息、金额等，以及可能需要提供交易记录或订单号。以下是一般的步骤：

1. 登录您的购物平台或联系商家，找到对应的订单。
2. 提供相关信息，告诉他们您需要补开发票。
3. 如果在线购买，他们可能会引导您填写申请表或者通过平台提供的客服请求。
4. 如果是实体店购买，可能需要携带相关凭证去店里进行申请。
5. 根据他们的流程，可能需要等待一段时间，他们会为您开具新的发票。

如果是在税务机关，您可能需要预约并携带相关材料到税务

In [6]:
evaluate(new_model ,  "你知道哪些快递公司？")

目前市面上知名的快递公司有顺丰速运、圆通速递、申通快递、中通快递、韵达快递、百世快递、京东物流、德邦物流、邮政EMS等。它们都是提供国内及国际快递服务的，其中顺丰以速度快、服务好而著称，而京东物流则更加注重电商配送和仓储服务。这些公司各有特色和优势，您可以根据自己的需求选择合适的快递公司。
-------
